In [2]:
import pandas as pd
from requests import get
import os
from pyaml_env import parse_config
from datetime import datetime
import psycopg2

In [3]:
API_KEY = parse_config('env.yml')['Environments'][0]['brapi_key']
DB_USER = parse_config('env.yml')['Environments'][0]['database_user']
DB_PW = parse_config('env.yml')['Environments'][0]['database_pw']
DB_NAME = parse_config('env.yml')['Environments'][0]['database_name']
DB_HOST = parse_config('env.yml')['Environments'][0]['database_url']
DB_PORT = parse_config('env.yml')['Environments'][0]['database_port']


In [31]:
def connect():
    params = {
        "dbname": "inv", /n /n /n
        "user": DB_USER,
        "password": DB_PW,
        "host": DB_HOST,
        "port": DB_PORT,
    }
    try:
        conn = psycopg2.connect(**params)
        return conn

    except (psycopg2.Error, Exception) as e:
        print(e)

In [44]:
class Stock:
    # TO DO 
    # CREATE A WAY THAT WHENEVER A NEW TRANSACTION IS DONE, IT FETCHES THE INFORMATION ABOUT IT'S STOCK (INFO, HISTORICALS, QUOTE)
    instances = []
    def __init__(self, ticker):
        self.ticker = ticker
        self.url = f'https://brapi.dev/api/quote/{self.ticker}'       
    
    def GetStockInfo(self):
        params = {
            'modules': 'summaryProfile',
            'token': API_KEY,}
        url = self.url
        try:
            req = get(url, params=params)
            data = req.json()
            df = pd.DataFrame(data['results'])
            info = pd.json_normalize(df['summaryProfile'])
            return info
        except Exception as e:
            print(e)

    def GetStockQuote(self):
        params = {
            'token': API_KEY}
        try:
            req = get(self.url, params=params)
            data = req.json()
            return pd.DataFrame(data['results'])
        except Exception as e:
            print(e)      

    def GetHistoricalQuote(self):
        params = {
            'token': API_KEY,
            'range':'3mo',
            'interval':'1d',}
        try:
            req = get(self.url, params=params)
            data = req.json()
            results = pd.json_normalize(data,  'results')
            df = pd.DataFrame(results['historicalDataPrice'][0])
            df['date'], df['ticker'] = df['date'].astype('datetime64[s]'), self.ticker
            return df
        except Exception as e:
            print(e)

    def TransformQuotes(data):   
        df = pd.DataFrame(data['results'])
        return df



In [28]:
class Transacao:
    instances = []
    def __init__(self, ticker: str, preco: float, quantidade: int, data: datetime.date = None):
        self.ticker = ticker
        self.price = preco
        self.quantity = quantidade
        self.date = data if data else datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        Transacao.instances.append(self)
        try:
            self.insert_into_db()
        except Exception as e:
            print(f"Error inserting transaction: {e}")

    def __repr__(self):
        return f"Transacao do {self.ticker} ao preço de {self.price} e {self.quantity} unidades, feita na data de'{self.date}'"


    def insert_into_db(self):
        with connect() as conn:
            with conn.cursor() as cur:
                insert_query = """
                INSERT INTO inv.public.transac(ticker, price, quantity, transac_date)
                VALUES (%s, %s, %s, %s)
                """
                cur.execute(insert_query, (self.ticker, self.price, self.quantity, self.date))
                conn.commit()
                
    @classmethod
    def get_from_backend(cls):
        cls.instances.clear()
        conn = connect()
        results = pd.read_sql_query("SELECT * FROM inv.public.transac", conn)
        for _, row in results.iterrows():
            Transacao(ticker=row['ticker'], 
                preco=row['price'],
                quantidade=row['quantity'],
                data=row['transac_date'])
        return Transacao.instances

In [ ]:
information_data = []
stocks = set([i.ticker for i in Transacao.instances])

for ticker in stocks:
    stock = Stock(ticker)
    data = stock.GetStockInfo()
    data['ticker'] = ticker
    information_data.append(data)
final = pd.concat(information_data)
final.head(20)





,address1,address2,city,state,zip,country,phone,website,industry,industryKey,industryDisp,sector,sectorKey,sectorDisp,longBusinessSummary,fullTimeEmployees,companyOfficers,irWebsite,executiveTeam,ticker
0,"Brigadeiro LuAs AntA nio Avenue, 1343",9th Floor,São Paulo,SP,01317-910,Brazil,55 11 3177 7014,https://www.ultra.com.br,Oil & Gas Refining & Marketing,oil-gas-refining-marketing,Oil & Gas Refining & Marketing,Energy,energy,Energy,"Ultrapar Participações S.A., through its subsi...",10126.0,[],http://www.ultra.com.br/ri/english/index.htm,[],UGPA3
0,Banco do Brasil Building,"3rd Floor North Wing, Block 05, Lot B, South T...",Brasília,DF,NaN,Brazil,NaN,https://www.bbseguridaderi.com.br,Insurance - Diversified,insurance-diversified,Insurance - Diversified,Financial Services,financial-services,Financial Services,"BB Seguridade Participações S.A., through its ...",222.0,[],NaN,[],BBSE3
0,Eldorado Business Tower,"30th floor Avenida Dra. Ruth Cardoso, No. 8,50...",São Paulo,SP,05425-070,Brazil,55 11 3040 6874,https://embraer.com,Aerospace & Defense,aerospace-defense,Aerospace & Defense,Industrials,industrials,Industrials,"Embraer S.A., together with its subsidiaries, ...",19179.0,[],http://ri.embraer.com.br/default.aspx?linguage...,[],EMBR3
0,EdifIcio Banco do Brasil,"Quadra 5, Lote B Autarquias Norte Federal Dist...",Brasília,DF,NaN,Brazil,55 80 0729 5285,https://www.bb.com.br,Banks - Regional,banks-regional,Banks - Regional,Financial Services,financial-services,Financial Services,"Banco do Brasil S.A., together with its subsid...",NaN,[],NaN,[],BBAS3
0,"Avenida Henrique Valadares, 28",19th floor,Rio De Janeiro,RJ,20241-030,Brazil,NaN,https://petrobras.com.br,Oil & Gas Integrated,oil-gas-integrated,Oil & Gas Integrated,Energy,energy,Energy,"Petróleo Brasileiro S.A. - Petrobras explores,...",46730.0,[],NaN,[],PETR4
0,"Parque Cidade Corporate Building, Tower A","Rooms 1.201, 1.202,1.204, 1.205 SCS, BLOCK 09,...",Brasília,DF,70308-200,Brazil,55 61 3426 1000,https://www.equatorialenergia.com.br,Utilities - Regulated Electric,utilities-regulated-electric,Utilities - Regulated Electric,Utilities,utilities,Utilities,"Equatorial Energia S.A., through its subsidiar...",NaN,[],NaN,[],EQTL3
0,"Av. Dra. Ruth Cardoso, 8501",8th floor,São Paulo,SP,05425-070,Brazil,55 11 3094 6300,https://www2.gerdau.com,Steel,steel,Steel,Basic Materials,basic-materials,Basic Materials,"Gerdau S.A., together with its subsidiaries, o...",NaN,[],NaN,[],GGBR3
0,"Rua da Quitanda, 196",9th Floor Centro,Rio De Janeiro,RJ,20091-005,Brazil,55 21 2514 4637,https://www.eletrobras.com,Utilities - Renewable,utilities-renewable,Utilities - Renewable,Utilities,utilities,Utilities,Centrais Elétricas Brasileiras S.A. - Eletrobr...,8328.0,[],NaN,[],ELET3
0,"Avenida Bernardo Vasconcelos, n° 377",Bairro Cachoeirinha,Belo Horizonte,MG,31150-000,Brazil,55 31 3247 7024,https://www.localiza.com,Rental & Leasing Services,rental-leasing-services,Rental & Leasing Services,Industrials,industrials,Industrials,Localiza Rent a Car S.A. engages in car and fl...,21091.0,[],NaN,[],RENT3
0,Praia de Botafogo 186,"offices 1101, 1701 and 1801 Botafogo",Rio De Janeiro,RJ,22250-145,Brazil,55 21 3485 5000,https://www.vale.com,Other Industrial Metals & Mining,other-industrial-metals-mining,Other Industrial Metals & Mining,Basic Materials,basic-materials,Basic Materials,"Vale S.A., together with its subsidiaries, pro...",NaN,[],NaN,[],VALE3


In [98]:
with connect() as conn:
    with conn.cursor() as cur:
        query = """
                INSERT INTO inv.public.stock_info(
                    address_one, 
                    address_two, 
                    city, 
                    state, 
                    zip, 
                    country, 
                    phone, 
                    industry, 
                    industry_key, 
                    industry_disp, 
                    sector, 
                    sector_key, 
                    sector_disp, 
                    bus_summary, 
                    employees, 
                    website, 
                    ticker  )
                VALUES (%s, %s, %s, %s,%s, %s, %s, %s,%s, %s, %s, %s,%s, %s, %s, %s,%s)""" 
        for row in final.itertuples():
                cur.execute(query, (row.address1, row.address2, row.city, row.state, row.zip, row.country, row.phone,  row.industry, row.industryKey, row.industryDisp, row.sector, row.sectorKey, row.sectorDisp, row.longBusinessSummary, row.fullTimeEmployees, row.irWebsite, row.ticker))
            
            